<a href="https://colab.research.google.com/github/modeverv/AI/blob/master/amateras_ikehata_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参照 
https://tech-blog.cloud-config.jp/2020-02-06-category-classification-using-bert/

In [7]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls /content/drive/My\ Drive/
!ls -la /content/drive/My\ Drive/bert/data/trains


 99_other	      bert		 drawio   横地チーム
 99_行動記録_コロナ  'Colab Notebooks'	 gyazo	  社員紹介
total 1
-rw------- 1 root root 182 Sep 14 05:13 features.csv
-rw------- 1 root root  33 Sep 14 05:13 labels.csv


In [9]:
# 必要ライブラリのインストール
!pip install sentencepiece
!pip install keras_bert
!pip install np_utils
# GPUとかTPUとか使いたい
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import os
import tensorflow_datasets as tfds
print(tf.__version__)

     |████████████████████████████████| 1.1MB 2.7MB/s 
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=e6f7d10da4d0286d04b1b524516fab514dc778a255d3873efc756725f109a453
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12942 sha256=bd689c722bcd969d4b75d00e5b9276270be88bb043bfd4d81dfb629a4b83114c
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=84ce67185898fc99824c403e37ea7ec2f30210de7ec4293a215db3a02153c5ca
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=cfb888f365afe1ed

In [10]:
# BERTの設定ファイル、モデルのロード
# max値を得るプログラム
import pandas as pd
import sentencepiece as spm

# feature.csvは上記で用意したファイルのパスを指定してください
train_features_df = pd.read_csv('/content/drive/My Drive/bert/data/trains/features.csv')

def _get_indice(feature):
    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))
    tokens.append('[SEP]')
    number = len(tokens)

    return number

sp = spm.SentencePieceProcessor()
# ダウンロードした事前学習モデルのパスを指定してください
sp.Load('/content/drive/My Drive/bert/bert-wiki-ja/wiki-ja.model')

numbers = []

for feature in train_features_df['feature']:
    features_number = _get_indice(feature)
    numbers.append(features_number)

# 最大トークン数
max_token_num = max(numbers)
print("max_token_number: " + str(max_token_num))

max_token_number: 18


In [13]:
# 学習データのロード関数
import sys
sys.path.append('modules')
from keras_bert import load_trained_model_from_checkpoint
from keras import utils

# BERTのロード
config_path = '/content/drive/My Drive/bert/bert-wiki-ja/bert_finetuning_config_v1.json'
# 拡張子まで記載しない
checkpoint_path = '/content/drive/My Drive/bert/bert-wiki-ja/model.ckpt-1400000'

# 最大のトークン数
SEQ_LEN = 18
BATCH_SIZE = 16
BERT_DIM = 768
LR = 1e-4
# 学習回数
EPOCH = 20 # 20

bert = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True,  trainable=True, seq_len=SEQ_LEN)
bert.summary()


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 18)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 18)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 18, 768), (3 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 18, 768)      1536        Input-Segment[0][0]              
_______________________________________________________________________________________

In [14]:
# 学習データのロード関数
from keras import utils
import numpy as np # これ必要だったよ？

maxlen = SEQ_LEN # maxlenがundefinedだよ？
sp = spm.SentencePieceProcessor()
sp.Load('/content/drive/My Drive/bert/bert-wiki-ja/wiki-ja.model')

def _get_indice(feature):
    indices = np.zeros((maxlen), dtype = np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(sp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = sp.piece_to_id(token)
        except:
            logging.warn(f'{token} is unknown.')
            indices[t] = sp.piece_to_id('<unk>')

    return indices

def _load_labeldata(train_dir, test_dir):
    train_features_df = pd.read_csv(f'{train_dir}/features.csv')
    train_labels_df = pd.read_csv(f'{train_dir}/labels.csv')
    test_features_df = pd.read_csv(f'{test_dir}/features.csv')
    test_labels_df = pd.read_csv(f'{test_dir}/labels.csv')
    label2index = {k: i for i, k in enumerate(train_labels_df['label'].unique())}
    index2label = {i: k for i, k in enumerate(train_labels_df['label'].unique())}
    class_count = len(label2index)
    train_labels = utils.np_utils.to_categorical([label2index[label] for label in train_labels_df['label']], num_classes=class_count)
    test_label_indices = [label2index[label] for label in test_labels_df['label']]
    test_labels = utils.np_utils.to_categorical(test_label_indices, num_classes=class_count)

    train_features = []
    test_features = []

    for feature in train_features_df['feature']:
        train_features.append(_get_indice(feature))
    train_segments = np.zeros((len(train_features), maxlen), dtype = np.float32)
    print("maxlen")
    print(maxlen)
    for feature in test_features_df['feature']:
        test_features.append(_get_indice(feature))
    test_segments = np.zeros((len(test_features), maxlen), dtype = np.float32)

    print(f'Trainデータ数: {len(train_features_df)}, Testデータ数: {len(test_features_df)}, ラベル数: {class_count}')

    return {
        'class_count': class_count,
        'label2index': label2index,
        'index2label': index2label,
        'train_labels': train_labels,
        'test_labels': test_labels,
        'test_label_indices': test_label_indices,
        'train_features': np.array(train_features),
        'train_segments': np.array(train_segments),
        'test_features': np.array(test_features),
        'test_segments': np.array(test_segments),
        'input_len': maxlen
    }

In [15]:
# モデル作成関数
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Flatten, GlobalMaxPooling1D
from keras_bert.layers import MaskedGlobalMaxPool1D
from keras import Input, Model
from keras_bert import AdamWarmup, calc_train_steps

def _create_model(input_shape, class_count):
    decay_steps, warmup_steps = calc_train_steps(
        input_shape[0],
        batch_size=BATCH_SIZE,
        epochs=EPOCH,
    )

    bert_last = bert.get_layer(name='NSP-Dense').output
    x1 = bert_last
    output_tensor = Dense(class_count, activation='softmax')(x1)
    # Trainableの場合は、Input Masked Layerが3番目の入力なりますが、
    # FineTuning時には必要無いので1, 2番目の入力だけ使用します。
    # Trainableでなければkeras-bertのModel.inputそのままで問題ありません。
    model = Model([bert.input[0], bert.input[1]], output_tensor)
    model.compile(loss='categorical_crossentropy',
                  optimizer=AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
                  #optimizer='nadam',
                  metrics=['mae', 'mse', 'acc'])

    return model

In [16]:
# 学習データのロードとモデルの準備
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.utils.np_utils import to_categorical
import numpy as np
from keras import utils

trains_dir = '/content/drive/My Drive/bert/data/trains'
tests_dir = '/content/drive/My Drive/bert/data/tests'

data = _load_labeldata(trains_dir, tests_dir)
model_filename = '/content/drive/My Drive/bert/models/knbc_finetuning.model'
model = _create_model(data['train_features'].shape, data['class_count'])

model.summary()

maxlen
18
Trainデータ数: 2, Testデータ数: 2, ラベル数: 2
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 18)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 18)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 18, 768), (3 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 18, 768)      1536        Input-Segment[0][0]              
__________________________________________

In [17]:
print(data)
#!gcloud beta services identity create --service tpu.googleapis.com --project $PROJECT_ID


{'class_count': 2, 'label2index': {'スポーツ': 0, '携帯電話': 1}, 'index2label': {0: 'スポーツ', 1: '携帯電話'}, 'train_labels': array([[1., 0.],
       [0., 1.]], dtype=float32), 'test_labels': array([[1., 0.],
       [0., 1.]], dtype=float32), 'test_label_indices': [0, 1], 'train_features': array([[    4,  4570,  4747,   162,    10,  4538,    11,   619,    63,
           92, 10636,   488,    12, 17862, 25046,   237,    27,     5],
       [    4,     9,  1778, 15448,  1035, 10301,   858,  8688,    10,
          977,    10, 16744,    95,  3017,     5,     0,     0,     0]],
      dtype=int32), 'train_segments': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]], dtype=float32), 'test_features': array([[    4,  4570,   488,    12, 17862, 25046,   237,    27,     5,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    4,     9,  1778,    12, 18695,

In [18]:
# 学習の実行
history = model.fit([data['train_features'], data['train_segments']],
          data['train_labels'],
          epochs = EPOCH,
          batch_size = BATCH_SIZE,
          validation_data=([data['test_features'], data['test_segments']], data['test_labels']),
          shuffle=False,
          verbose = 1,
          callbacks = [
              ModelCheckpoint(monitor='val_acc', mode='max', filepath=model_filename, save_best_only=True)
          ])


Epoch 1/20
1/1 [==============================] - ETA: 0s - loss: 0.9856 - mae: 0.5700 - mse: 0.3705 - acc: 0.5000WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/bert/models/knbc_finetuning.model/assets
1/1 [==============================] - 36s 36s/step - loss: 0.9856 - mae: 0.5700 - mse: 0.3705 - acc: 0.5000 - val_loss: 0.4456 - val_mae: 0.3595 - val_mse: 0.1293 - val_acc: 1.0000
Epoch 2/20
1/1 [==============================] - 0s 31ms/step - loss: 0.2423 - mae: 0.2147 - mse: 0.0469 - acc: 1.0000 - val_loss: 

In [22]:
df = pd.DataFrame(history.history)
display(df)

,loss,mae,mse,acc,val_loss,val_mae,val_mse,val_acc
0,0.985641,0.570027,3.705293e-01,0.5,0.445565,0.359532,0.129270,1.0
1,0.242337,0.214711,4.688192e-02,1.0,0.413619,0.303148,0.140246,0.5
2,0.041628,0.040772,1.664603e-03,1.0,0.484121,0.322187,0.183485,0.5
3,0.009095,0.009053,8.347549e-05,1.0,0.530710,0.334473,0.208834,0.5
4,0.007576,0.007547,5.697853e-05,1.0,0.552353,0.339151,0.220437,0.5
5,0.004617,0.004607,2.141487e-05,1.0,0.560070,0.340137,0.224878,0.5
6,0.003022,0.003017,1.006385e-05,1.0,0.555508,0.337715,0.223449,0.5
7,0.001929,0.001927,3.800122e-06,1.0,0.549490,0.335141,0.221146,0.5
8,0.000713,0.000713,5.087019e-07,1.0,0.542704,0.332496,0.218351,0.5
9,0.000456,0.000456,2.317388e-07,1.0,0.536242,0.330057,0.215604,0.5


In [23]:
# モデルの評価
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model
from keras_bert import get_custom_objects

model = load_model(model_filename, custom_objects=get_custom_objects())

predicted_test_labels = model.predict([data['test_features'], data['test_segments']]).argmax(axis=1)
numeric_test_labels = np.array(data['test_labels']).argmax(axis=1)

report = classification_report(
        numeric_test_labels, predicted_test_labels, target_names=['スポーツ','携帯電話'], output_dict=True)
display(pd.DataFrame(report).T)

,precision,recall,f1-score,support
スポーツ,1.0,1.0,1.0,1.0
携帯電話,1.0,1.0,1.0,1.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,2.0
weighted avg,1.0,1.0,1.0,2.0


In [24]:
# 予測
import sys
import pandas as pd
import sentencepiece as spm
import logging
import numpy as np

from keras import utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import get_custom_objects
from sklearn.metrics import classification_report, confusion_matrix

sys.path.append('modules')

# SentencePieceProccerモデルの読込
spp = spm.SentencePieceProcessor()
spp.Load('/content/drive/My Drive/bert/bert-wiki-ja/wiki-ja.model')
# BERTの学習したモデルの読込
model_filename = '/content/drive/My Drive/bert/models/knbc_finetuning.model'
model = load_model(model_filename, custom_objects=get_custom_objects())

SEQ_LEN = 18
maxlen = SEQ_LEN

def _get_indice(feature):
    indices = np.zeros((maxlen), dtype=np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(spp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = spp.piece_to_id(token)
        except:
            logging.warn('unknown')
            indices[t] = spp.piece_to_id('<unk>')
    return indices


In [40]:
# 予測実行
feature = "スポーツ"

test_features = []
test_features.append(_get_indice(feature))
test_segments = np.zeros(
    (len(test_features), maxlen), dtype=np.float32)

predicted_test_labels = model.predict(
    [np.array(test_features), test_segments]).argmax(axis=1)
label_data = pd.read_csv('/content/drive/My Drive/bert/label_id/id_category.csv')
label = label_data.query(f'id == {predicted_test_labels[0]}')
label = label.iloc[0]
label_name = label['label']
print(label_name)


携帯電話
